In [ ]:
# TODO: create another dictionary with nd2 average fluorescence; extract it for each session.
# TODO: save the extracted data in a h5 file or something similar. Do session extraction for whole dataset

In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
import labrotation.file_handling as fh
import h5py
from time import time
import matplotlib.pyplot as plt
import numpy as np
import os
from labrotation import file_handling as fh
from copy import deepcopy
import pandas as pd
import labrotation.two_photon_session as tps
import seaborn as sns
import uuid  # for unique labeling of sessions and coupling arrays (mouse velocity, distance, ...) to sessions in dataframe 
from matplotlib import cm  # colormap
import datadoc_util

# If exists, load environmental variables from .env file

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

# Set up data documentation directory

In [ ]:
# assumption: inside the documentation folder, the subfolders carry the id of each mouse (not exact necessarily, but they 
# can be identified by the name of the subfolder). 
# Inside the subfolder xy (for mouse xy), xy_grouping.xlsx and xy_segmentation.xlsx can be found.
# xy_grouping.xlsx serves the purpose of finding the recordings belonging together, and has columns:
# folder, nd2, labview, lfp, face_cam_last, nikon_meta, experiment_type, day
# xy_segmentation.xlsx contains frame-by-frame (given by a set of disjoint intervals forming a cover for the whole recording) 
# classification of the events in the recording ("normal", seizure ("sz"), sd wave ("sd_wave") etc.). The columns:
# folder, interval_type, frame_begin, frame_end.

# TODO: write documentation on contents of xlsx files (what the columns are etc.)
if "DATA_DOCU_FOLDER" in env_dict.keys():
    docu_folder = env_dict["DATA_DOCU_FOLDER"]
else:
    docu_folder = fh.open_dir("Choose folder containing folders for each mouse!")
print(f"Selected folder:\n\t{docu_folder}")

In [ ]:
if "documentation" in os.listdir(docu_folder):
    mouse_folder = os.path.join(docu_folder, "documentation")
else:
    mouse_folder = docu_folder
mouse_names = os.listdir(mouse_folder)
print(f"Mice detected:")
for mouse in mouse_names:
    print(f"\t{mouse}")

### Load matlab-2p

In [ ]:
if "MATLAB_2P_FOLDER" in env_dict.keys():
    matlab_2p_folder = env_dict["MATLAB_2P_FOLDER"]
else:
    matlab_2p_folder = fh.open_dir("Choose matlab-2p folder")
print(f"matlab-2p folder set to:\n\t{matlab_2p_folder}")

In [ ]:
df_seg_complete = pd.DataFrame(columns = ["nd2", "interval_type", "frame_begin", "frame_end"])
df_grouping_complete = pd.DataFrame(columns = ["folder", "nd2", "labview", "lfp", "face_cam_last", "nikon_meta", "experiment_type", "mouse_id", "day"])

for mouse_id in mouse_names:
    print(mouse_id)
    seg_fpath = os.path.join(mouse_folder, mouse_id, mouse_id + '_segmentation.xlsx')
    grouping_fpath = os.path.join(mouse_folder, mouse_id, mouse_id + '_grouping.xlsx')
    if os.path.exists(seg_fpath) and os.path.exists(grouping_fpath):
        df_seg = pd.read_excel(seg_fpath)
        df_grouping = pd.read_excel(grouping_fpath)
        df_grouping["mouse_id"] = mouse_id
        # select only tmev, chr2_szsd, chr2_sd, chr2_ctl experiment data first
        df_grouping = df_grouping[df_grouping["experiment_type"].isin(["tmev", "tmev_ctl", "chr2_sd", "chr2_szsd", "chr2_ctl"])]
        # merge into large dataframes
        # print(f"\tseg bef: {len(df_seg_complete['nd2'])}")
        df_seg_complete = pd.concat([df_seg_complete, df_seg])
        # print(f"\tseg aft: {len(df_seg_complete['nd2'])}")
        # print(f"\tgro bef: {len(df_grouping_complete['nd2'])}")
        df_grouping_complete = pd.concat([df_grouping_complete, df_grouping])
        # print(f"\tgro aft: {len(df_grouping_complete['nd2'])}")
    else:
        print(f"Check if you set the correct folder (folder containing all subfolders with mouse names):")
        if not os.path.exists(seg_fpath):
            print(f"\t{seg_fpath} not found")
        if not os.path.exists(grouping_fpath):
            print(f"\t{grouping_fpath} not found")
        

In [ ]:
docu_folder

In [ ]:
ddoc = datadoc_util.DataDocumentation(docu_folder)
ddoc.loadDataDoc()

In [ ]:
# take only recordings that were classified as "tmev" (experiment type)
df_seg_complete.where(df_seg_complete["nd2"].isin(df_grouping_complete["nd2"].unique()), inplace=True)
# wrong recording types changed to NaN; drop them
df_seg_complete.dropna(inplace=True)

In [ ]:
# need to create new dataframe. 
# x axis will be: baseline, seizure, sd_wave, sd_extinction, aftermath...
# one plot: only with seizures 
#   (check categories: chr2_sz/chr2_szsd? tmev_sz? check interval types corresponding to seizure, sd etc.)
# other plot: chr2 only?

# Baseline - Seizure - SD - aftermath analysis 

In [ ]:
# take only videos with seizure
df_sz = df_seg_complete.groupby("nd2").filter(lambda group: "sz" in group["interval_type"].unique())

In [ ]:
len(df_sz)

## Add uuid

In [ ]:
df_sz = pd.merge(df_sz, ddoc.getNikonFileNameUuid().dropna(), on="nd2")

## Add seizure-uuid
See 4 Directionality analysis for original context. This code should be the same as there! (If update needed, need to extract into third file!)
In this analysis, the purpose is to make each seizure unique (to deal with seizures split-up in two videos, for example)

In [ ]:
df_sz["uuid_sz"] = df_sz["uuid"]
# following two recordings contain 1 seizure-sd event
df_sz["uuid_sz"] = df_sz["uuid_sz"].replace("65bff16a4cf04930a5cb14f489a8f99b", "30dc55d1a5dc4b0286d132e72f208ca6")
# following recordings do not have sz
#qdf = qdf[qdf["uuid_matched"] != "171693d0988c458a96c8198c7b8cfc28"]

In [ ]:
df_sz[df_sz["uuid_sz"] == "30dc55d1a5dc4b0286d132e72f208ca6"]

# Old analysis

### TODO: some recordings start with non-"normal" event type! How to deal with these? E.g. fake-handling!
### TODO: test if _nik.txt is read correctly for stimulations (where first column has weird format)!

In [ ]:
# change labels of segments that start the video until seizure as "baseline"
df_sz.loc[(df_sz["interval_type"] == "normal") & (df_sz["frame_begin"] == 1.0), "interval_type"] = "baseline"
# assuming one seizure per video, and that there is no "normal" labeling between "abnormal" events like SZ and SD, 
#   change label to aftermath
# TODO: change this to frame_end == len(recording)? i.e. last part in video?
df_sz.loc[(df_sz["interval_type"] == "normal") & (df_sz["frame_begin"] != 1.0), "interval_type"] = "aftermath"  # second criterion is not even necessary anymore...

In [ ]:
# TODO: some recordings have iis between "normal". Change them to normal to include them in movement statistics.

In [ ]:
len(df_grouping_complete["nd2"]) == len(df_grouping_complete["nd2"].unique())  # grouping should be unique!

In [ ]:
sz_vids = df_sz["nd2"].unique()
df_sz_grouping = df_grouping_complete[df_grouping_complete["nd2"].isin(sz_vids)]

In [ ]:
len(df_sz_grouping["nd2"]) == len(df_sz_grouping["nd2"].unique())

In [ ]:
df_sz_grouping.keys()

In [ ]:
tps.from_hdf5()  # TODO: try opening sessions. If works properly, adjust script below to take hdf5 files

In [ ]:
# TODO: rename row to row_grouping, row2 to row_segment

sz_nd2s = list()  # should contain nd2 file names for backtracking
sz_segments_speed_dict = dict()
sz_segments_dist_dict = dict()
sz_segments_totdist_dict = dict()
sz_segments_rounds_dict = dict()
sz_segments_tsscn_dict = dict()
sz_segments_running_dict = dict()

df_sz_res_cols = {"mouse_id": [], "experiment_type": [], "interval_type": [], "distance": [], "d_frames": [], "dt": []}

# for each recording in category, create session, 
# perform labview cut to scanner timeframe, 
# then extract relevant frame movement data
# results appended to segments_dict entries.
# segments_dict should contain, for each key (type of interval) all the corresponding data from all videos
for index, row in df_sz_grouping.iterrows():
    labview_fpath = os.path.join(row["folder"], row["labview"])
    nikmeta_fpath = os.path.join(row["folder"], row["nikon_meta"])
    #TODO: instead of running the functions below, use init_and_process() constructor!
    ses = tps.TwoPhotonSession(nd2_timestamps_path=nikmeta_fpath, labview_path=labview_fpath, matlab_2p_folder=matlab_2p_folder)
    ses.infer_labview_timestamps()
    ses._open_data()  # TODO: implement a better way than using "private" function.
    ses_segments = df_sz[df_sz["nd2"]==row["nd2"]]  # get all segments from the same file
    for index2, row2 in ses_segments.iterrows():
        t0 = int(row2["frame_begin"] - 1)  # correct 1-indexing to 0-indexing, convert to int
        t1 = int(row2["frame_end"])  # list[a:b] returns elements a to (b-1), so no need to subtract here
        
        # Check if dictionary entries exist
        if row2["interval_type"] not in sz_segments_speed_dict.keys():
            sz_segments_speed_dict[row2["interval_type"]] = []
        if row2["interval_type"] not in sz_segments_dist_dict.keys():   
            sz_segments_dist_dict[row2["interval_type"]] = []
        if row2["interval_type"] not in sz_segments_totdist_dict.keys():     
            sz_segments_totdist_dict[row2["interval_type"]] = []
        if row2["interval_type"] not in sz_segments_rounds_dict.keys():    
            sz_segments_rounds_dict[row2["interval_type"]] = []
        if row2["interval_type"] not in sz_segments_tsscn_dict.keys():    
            sz_segments_tsscn_dict[row2["interval_type"]] = []
        if row2["interval_type"] not in sz_segments_running_dict.keys():    
            sz_segments_running_dict[row2["interval_type"]] = []
        
        # append to dictionaries and nd2 list
        sz_nd2s.append(row["nd2"])
        sz_segments_speed_dict[row2["interval_type"]].append(deepcopy(ses.belt_scn_dict["speed"][t0:t1]))
        sz_segments_dist_dict[row2["interval_type"]].append(deepcopy(ses.belt_scn_dict["distance"][t0:t1]))
        sz_segments_totdist_dict[row2["interval_type"]].append(deepcopy(ses.belt_scn_dict["totdist"][t0:t1]))
        sz_segments_rounds_dict[row2["interval_type"]].append(deepcopy(ses.belt_scn_dict["rounds"][t0:t1]))
        sz_segments_tsscn_dict[row2["interval_type"]].append(deepcopy(ses.belt_scn_dict["tsscn"][t0:t1]))
        sz_segments_running_dict[row2["interval_type"]].append(deepcopy(ses.belt_scn_dict["running"][t0:t1]))
        
        # append to dataframe columns
        df_sz_res_cols["mouse_id"].append(row["mouse_id"])
        df_sz_res_cols["experiment_type"].append(row["experiment_type"])
        df_sz_res_cols["interval_type"].append(row2["interval_type"])
        df_sz_res_cols["distance"].append(ses.belt_scn_dict["totdist"][t1-1] - ses.belt_scn_dict["totdist"][t0])  # t1 is 1-indexed, t0 0-indexed, see above
        df_sz_res_cols["d_frames"].append(t1 - 1 - t0)  # t1 is 1-indexed, t0 0-indexed, see above
        # dt: in ms
        df_sz_res_cols["dt"].append(ses.belt_scn_dict["tsscn"][t1-1] - ses.belt_scn_dict["tsscn"][t0]) # t1 is 1-indexed, t0 0-indexed, see above

In [ ]:
# create df from dict:
df_sz_res = pd.DataFrame.from_dict(df_sz_res_cols)

In [ ]:
# normalize distance by the interval length 
# also take absolute value; especially during shorter intervals, mice might step backwards a few times and that's it 
df_sz_res["dist_norm"] = abs(df_sz_res["distance"]/df_sz_res["dt"])  

In [ ]:
df_sz_res.head()

In [ ]:
df_sz_res["mouse_id"].unique()

In [ ]:
# todo work with dataframe, seaborn plot. if fine, use chr2 too (ctl stim vs szsd, vs sd only. need more sd only?)

In [ ]:
# 3. open 2psession for each nd2, do matching. Need nd2? or only nd2 meta + labview. Latter would be much quicker.

In [ ]:
# TODO: create df from results! With ID, exp type, interval type...
# TODO: visualize data somehow. Take code from below (Unorganized)!

In [ ]:
# result dataframe could be:
# ID, interval_type, distance, dt, distance_norm

In [ ]:
df_sz_res["interval_type"].unique()

In [ ]:
df_sz_res["experiment_type"].unique()

In [ ]:
# TODO: convert iis to normal for the sake of locomotion analysis?
# TODO: add more recordings with seizures. Don't need complete tmev groupings for now... Only videos with seizures.

In [ ]:
df_sz_res["interval_type_renamed"] = df_sz_res["interval_type"].replace({"sd_wave_cx": "sd_wave"})
df_sz_res["experiment_type_renamed"] = df_sz_res["experiment_type"].replace({"tmev": "TMEV", "chr2_szsd": "ChR2"})

# Save dataframes

In [ ]:
df_output_dir = fh.open_dir("Choose folder for saving dataframe")
df_sz_res_fname = fh.get_filename_with_date("loco_analysis", extension='.h5')
df_out_fpath = os.path.join(df_output_dir, df_sz_res_fname)

df_sz_res.to_hdf(df_out_fpath, key='df_sz_res', mode='a')
df_seg.to_hdf(df_out_fpath, key='df_seg', mode='a')
df_seg_complete.to_hdf(df_out_fpath, key='df_seg_complete', mode='a')

### Check for consistency after saving

In [ ]:
# df_sz_res2 = pd.read_hdf(df_out_fpath, key='df_sz_res')
# df_seg2= pd.read_hdf(df_out_fpath, key='df_seg')
# df_seg_complete2 = pd.read_hdf(df_out_fpath, key='df_seg_complete')

In [ ]:
sns.set_theme(style="whitegrid")
#sns.set(rc={'figure.figsize':(20.7,8.27)})

# use standard error or standard deviation for uncertainty/spread, or confidence interval or percentile interval
# se/sd/ci/pi
# see https://seaborn.pydata.org/tutorial/error_bars.html
g = sns.catplot(
    data=df_sz_res, kind="bar",
    x="interval_type_renamed", y="dist_norm", hue="experiment_type_renamed",
    errorbar=("pi", 50), alpha=.8, height=12, aspect=1.3
);
g.despine(left=True)
g.set_axis_labels("", "(distance) / (interval length)", fontsize=22)
g.set_xticklabels(["Baseline", "Seizure", "SD Wave", "SD Extinction", "Aftermath", "IIS", "Stimulation"], fontsize=18)
g.set_yticklabels(fontsize=18)
g._legend.set_title("Experiment type")
plt.setp(g._legend.get_title(), fontsize=20);
plt.setp(g._legend.get_texts(),  fontsize=18);

In [ ]:
sns.set_theme(style="whitegrid")
#sns.set(rc={'figure.figsize':(20.7,8.27)})

# use standard error or standard deviation for uncertainty/spread, or confidence interval or percentile interval
# se/sd/ci/pi
# see https://seaborn.pydata.org/tutorial/error_bars.html
g = sns.catplot(
    data=df_sz_res, kind="strip",
    x="interval_type_renamed", y="dist_norm", hue="experiment_type_renamed",
    errorbar=("pi", 50), alpha=0.8, height=12, aspect=1.3, size=12
);
g.despine(left=True)
g.set_axis_labels("", "(distance) / (interval length)", fontsize=22)
g.set_xticklabels(["Baseline", "Seizure", "SD Wave", "SD Extinction", "Aftermath", "IIS", "Stimulation"], fontsize=18)
g.set_yticklabels(fontsize=18)
g._legend.set_title("Experiment type")
plt.setp(g._legend.get_title(), fontsize=20);
plt.setp(g._legend.get_texts(),  fontsize=18);
g.set(ylim=(-0.003,0.06));

In [ ]:
len(df_sz_res[(df_sz_res["interval_type_renamed"] == "baseline") & (df_sz_res["experiment_type"] == "tmev")])

# Compare ChR2 SD vs SZSD running statistics

In [ ]:
# take only videos with seizure
df_stim = df_seg_complete.groupby("nd2").filter(lambda group: "stimulation" in group["interval_type"].unique())

In [ ]:
# change labels of segments that start the video until seizure as "baseline"
df_stim.loc[(df_stim["interval_type"] == "normal") & (df_stim["frame_begin"] == 1.0), "interval_type"] = "baseline"
# assuming one seizure per video, and that there is no "normal" labeling between "abnormal" events like SZ and SD, 
#   change label to aftermath
# TODO: change this to frame_end == len(recording)? i.e. last part in video?
df_stim.loc[(df_stim["interval_type"] == "normal") & (df_stim["frame_begin"] != 1.0), "interval_type"] = "aftermath"  # second criterion is not even necessary anymore...

In [ ]:
stim_vids = df_stim["nd2"].unique()
df_stim_grouping = df_grouping_complete[df_grouping_complete["nd2"].isin(stim_vids)]

In [ ]:
len(df_stim_grouping["nd2"]) == len(df_stim_grouping["nd2"].unique())

In [ ]:
# TODO: integrate this in sz for loop for faster re-run of notebook

In [ ]:
ses_segments

In [ ]:
# TODO: rename row to row_grouping, row2 to row_segment

stim_segments_speed_dict = dict()
stim_segments_dist_dict = dict()
stim_segments_totdist_dict = dict()
stim_segments_rounds_dict = dict()
stim_segments_tsscn_dict = dict()
stim_segments_running_dict = dict()

df_stim_res_cols = {"mouse_id": [], "experiment_type": [], "interval_type": [], "distance": [], "d_frames": [], "dt": [], "uid": []}

# for each recording in category, create session, 
# perform labview cut to scanner timeframe, 
# then extract relevant frame movement data
# results appended to segments_dict entries.
# segments_dict should contain, for each key (type of interval) all the corresponding data from all videos
for index, row in df_stim_grouping.iterrows():
    labview_fpath = os.path.join(row["folder"], row["labview"])
    nikmeta_fpath = os.path.join(row["folder"], row["nikon_meta"])
    
    ses = tps.TwoPhotonSession(nd2_timestamps_path=nikmeta_fpath, labview_path=labview_fpath, matlab_2p_folder=matlab_2p_folder)
    ses.infer_labview_timestamps()
    ses._open_data()  # TODO: implement a better way than using "private" function
    ses_segments = df_stim[df_stim["nd2"]==row["nd2"]]  # get all segments from the same file
    for index2, row2 in ses_segments.iterrows():
        uid = uuid.uuid1()  # unique ID of segment, for matching segment labview data with dataframe row
        t0 = int(row2["frame_begin"] - 1)  # correct 1-indexing to 0-indexing, convert to int
        t1 = int(row2["frame_end"])  # list[a:b] returns elements a to (b-1), so no need to subtract here
        
        # Check if dictionary entries exist
        #if row2["interval_type"] not in stim_segments_speed_dict.keys():
        #    stim_segments_speed_dict[row2["interval_type"]] = []
        #if row2["interval_type"] not in stim_segments_dist_dict.keys():   
        #    stim_segments_dist_dict[row2["interval_type"]] = []
        #if row2["interval_type"] not in stim_segments_totdist_dict.keys():     
        #    stim_segments_totdist_dict[row2["interval_type"]] = []
        #if row2["interval_type"] not in stim_segments_rounds_dict.keys():    
        #    stim_segments_rounds_dict[row2["interval_type"]] = []
        #if row2["interval_type"] not in stim_segments_tsscn_dict.keys():    
        #    stim_segments_tsscn_dict[row2["interval_type"]] = []
        #if row2["interval_type"] not in stim_segments_running_dict.keys():    
        #    stim_segments_running_dict[row2["interval_type"]] = []
        
        # append to dictionaries and nd2 list
        stim_segments_speed_dict[uid.int] = deepcopy(ses.belt_scn_dict["speed"][t0:t1])
        stim_segments_dist_dict[uid.int] = deepcopy(ses.belt_scn_dict["distance"][t0:t1])
        stim_segments_totdist_dict[uid.int] = deepcopy(ses.belt_scn_dict["totdist"][t0:t1])
        stim_segments_rounds_dict[uid.int] = deepcopy(ses.belt_scn_dict["rounds"][t0:t1])
        stim_segments_tsscn_dict[uid.int] = deepcopy(ses.belt_scn_dict["tsscn"][t0:t1])
        stim_segments_running_dict[uid.int] = deepcopy(ses.belt_scn_dict["running"][t0:t1])
        
        # append to dataframe columns
        df_stim_res_cols["mouse_id"].append(row["mouse_id"])
        df_stim_res_cols["experiment_type"].append(row["experiment_type"])
        df_stim_res_cols["interval_type"].append(row2["interval_type"])
        df_stim_res_cols["distance"].append(ses.belt_scn_dict["totdist"][t1-1] - ses.belt_scn_dict["totdist"][t0])  # t1 is 1-indexed, t0 0-indexed, see above
        df_stim_res_cols["d_frames"].append(t1 - 1 - t0)  # t1 is 1-indexed, t0 0-indexed, see above
        # dt: in ms
        df_stim_res_cols["dt"].append(ses.belt_scn_dict["tsscn"][t1-1] - ses.belt_scn_dict["tsscn"][t0]) # t1 is 1-indexed, t0 0-indexed, see above
        df_stim_res_cols["uid"].append(uid.int)

In [ ]:
# create df from dict:
df_stim_res = pd.DataFrame.from_dict(df_stim_res_cols)

In [ ]:
df_stim_res.head()

In [ ]:
# normalize distance by the interval length 
# also take absolute value; especially during shorter intervals, mice might step backwards a few times and that's it 
df_stim_res["dist_norm"] = abs(df_stim_res["distance"]/df_stim_res["dt"])  

In [ ]:
df_stim_res["dist_norm"].describe()

In [ ]:
df_stim_res.head()

In [ ]:
df_stim_res["experiment_type"].unique()

In [ ]:
df_stim_res["experiment_type_renamed"] = df_stim_res["experiment_type"].replace({"chr2_ctl": "Control", "chr2_szsd": "Sz+SD", "chr2_sd": "SD"})

In [ ]:
df_stim_res.head()

In [ ]:
sns.set_theme(style="whitegrid")
# use standard error or standard deviation for uncertainty/spread, or confidence interval or percentile interval
# se/sd/ci/pi
# see https://seaborn.pydata.org/tutorial/error_bars.html

g = sns.catplot(
    data=df_stim_res, kind="bar",
    x="interval_type", y="distance", hue="experiment_type_renamed",
    errorbar=("pi", 50), height=12, aspect=1.3 #, alpha=0.6,
)

#g = sns.catplot(
#    data=df_stim_res, kind="box",
#    x="interval_type", y="dist_norm", hue="experiment_type",
#    errorbar="se", palette="dark", height=10, aspect=1.3
#)
#sns.stripplot(
#    data=df_stim_res,
#    x="interval_type", y="dist_norm", hue="experiment_type", size=12,
#)

g.despine(left=True)
g.set_axis_labels("", "(distance) / (interval length)", fontsize=22)
#g.set_xticklabels(["Baseline", "Stimulation", "Aftermath", "SD extinciton", "Seizure", "SD Wave",], fontsize=18)
g.set_xticklabels(fontsize=18)
g.set_yticklabels(fontsize=18)
g._legend.set_title("Experiment type")
plt.setp(g._legend.get_title(), fontsize=20);
plt.setp(g._legend.get_texts(),  fontsize=18);

In [ ]:
g2 = sns.catplot(
    data=df_stim_res, kind="strip",
    x="interval_type", y="dist_norm", hue="experiment_type_renamed", height=12, aspect=1.3, size=12
)
g2.despine(left=True)
g2.set_axis_labels("", "(distance) / (interval length)", fontsize=22)
#g2.set_xticklabels(["Baseline", "Stimulation", "Aftermath", "Seizure", "SD Wave", "SD Extinction"], fontsize=18)
g2.set_xticklabels( fontsize=18)
g2.set_yticklabels(fontsize=18)
g2._legend.set_title("Experiment type")
plt.setp(g2._legend.get_title(), fontsize=20);
plt.setp(g2._legend.get_texts(),  fontsize=18);
g2.set(ylim=(-0.01,0.16));

# Plot individual sessions over each other

In [ ]:
df_stim_res

In [ ]:
#  make a plot of individual movement 
g_exptype_itype = df_stim_res.groupby(["experiment_type", "interval_type"])

In [ ]:
bl_ctl = g_exptype_itype.get_group(("chr2_ctl", "baseline"))["uid"]
bl_sd  = g_exptype_itype.get_group(("chr2_sd", "baseline"))["uid"]
bl_szsd= g_exptype_itype.get_group(("chr2_szsd","baseline"))["uid"]

In [ ]:
bl_ctl_tsscn = dict()
for uid_ctl in bl_ctl:
    bl_ctl_tsscn[uid_ctl] = stim_segments_tsscn_dict[uid_ctl]

In [ ]:
t_min =  9000000000  # will be first recorded time point in subset of data
t_max = -9000000000  # latest time point in subset of data
for key, value in bl_ctl_tsscn.items():
    if min(value) < t_min:
        t_min = min(value)
    if max(value) > t_max:
        t_max = max(value)

In [ ]:
bl_ctl = sorted(bl_ctl, key= lambda uid: sum(stim_segments_running_dict[uid]), reverse=True)

In [ ]:
# match recordings in time by shifting so that last data point has the same time in all cases
stim_segments_t_matched_dict = dict()
for uid in bl_ctl:
    dt = t_max - stim_segments_tsscn_dict[uid][-1]
    stim_segments_t_matched_dict[uid] = dt + deepcopy(stim_segments_tsscn_dict[uid])

In [ ]:
stim_segments_tsscn_dict[bl_ctl[0]] 

In [ ]:
cmap = cm.get_cmap('viridis', len(bl_ctl))
fig = plt.figure(figsize=(18,24))
for i_session, ctl_session in enumerate(bl_ctl):
    plt.plot(stim_segments_t_matched_dict[ctl_session],i_session*0.7 + stim_segments_speed_dict[ctl_session], color=cmap.colors[i_session])
plt.show()

In [ ]:
# TODO: create a UID for individual recordings (to match various segments in the dictionaries).
# TODO: plot movement starting with stimulation. How long it should be plotted? Match color map with baseline,
#       sort by movement amount 

In [ ]:
cmap = cm.get_cmap('viridis', len(bl_ctl))
fig = plt.figure(figsize=(18,18))
for i_session, ctl_session in enumerate(bl_ctl):
    plt.plot(i_session*0.5 + stim_segments_speed_dict[ctl_session], color=cmap.colors[i_session])
plt.show()

In [ ]:
df_stim_res.head()

In [ ]:
len(df_stim_res[(df_stim_res["experiment_type"] == "chr2_sd") & (df_stim_res["interval_type"] == "baseline")])

# TMEV running over days

# Unorganized

In [ ]:
# keys of belt_scn_dict: 'tsscn', 'rounds', 'speed', 'distance', 'totdist', 'running'

In [ ]:
# think about data to quantify (integrated velocity/distance per interval? divided by interval length)
# maybe need nd2 to extract mean, check if zones correspond to imaging data

In [ ]:
res_dict = dict()
for interval_type in segments_totdist_dict.keys():
    res_dict[interval_type] = []
    for dist_lis in segments_totdist_dict[interval_type]:
        assert dist_lis[-1] >= 0
        assert dist_lis[0] >= 0
        totdist_interval = (dist_lis[-1] - dist_lis[0])/len(dist_lis)
        res_dict[interval_type].append(totdist_interval)

In [ ]:
res_dict.keys()

In [ ]:
# TODO: get results also for control animal, make a dataframe including all these 
# (columns: 
#    tmev/chr2/ctl (exp_type?), 
#    interval_type ('baseline', 'sz', 'sd_wave', 'sd_extinction', 'normal'),
#    total distance,
#    etc. (other data like dist, for comparison. But I think totdist is the one I need)

# Then plot as seaborn catplot (grouped barplot)

In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid")


# Draw a nested barplot by species and sex
g = sns.catplot(
    data=a, kind="bar",
    x="", y="body_mass_g", hue="sex",
    errorbar="sd", palette="dark", alpha=.6, height=6
)
g.despine(left=True)
g.set_axis_labels("", "Body mass (g)")
g.legend.set_title("")